In [42]:
%%bash
docker run \
  --rm \
  --name omnisci_test \
  --ipc=host \
  -d --runtime=nvidia \
  -v $(readlink -f ..)/data:/data \
  -v $(readlink -f ..)/scripts:/src \
  -p 6273-6280:6273-6280 \
  omnisci/core-os-cuda:v5.2.1

3a361389908bf0f2ca7200c5ada3b47f1dba385c86e9d9d6d0d773f9eca5b78e


In [43]:
%%bash
docker exec omnisci_test /bin/bash -c \
    "cat /src/load_dataset.sql | /omnisci/bin/omnisql -p 'HyperInteractive' -t"

User admin connected to database omnisci
Result
Loaded: 1067371 recs, Rejected: 0 recs in 0.440000 secs
1 rows returned.
Execution time: 545 ms, Total time: 546 ms
User admin disconnected from database omnisci


In [8]:
%%bash
docker exec omnisci_test /bin/bash -c \
    "echo 'SELECT Invoice, SUM(Price) as price_sum FROM online_retail GROUP BY Invoice ORDER BY price_sum DESC LIMIT 10;' | /omnisci/bin/omnisql -p 'HyperInteractive' -t"

In [44]:
%%bash
docker exec omnisci_test /bin/bash -c \
    "echo 'SELECT * FROM online_retail LIMIT 10;' | /omnisci/bin/omnisql -p 'HyperInteractive' -t"

User admin connected to database omnisci
Invoice|StockCode|Description|Quantity|InvoiceDate|Price|CustomerID|Country
490286|21706|FOLDING UMBRELLA , RED/WHITE SPOT|1|2009-12-04 12:59:00|4.95|14527|United Kingdom
490286|85123A|WHITE HANGING HEART T-LIGHT HOLDER|6|2009-12-04 12:59:00|2.95|14527|United Kingdom
490286|84828|JUNGLE POPSICLES ICE LOLLY HOLDERS|6|2009-12-04 12:59:00|1.25|14527|United Kingdom
491041|90003D|CRYSTAL PAIR HEART HAIR SLIDES|1|2009-12-09 09:42:00|3.9|NULL|United Kingdom
490353|22042|CHRISTMAS CARD SINGING ANGEL|12|2009-12-04 16:37:00|0.42|15514|United Kingdom
490286|35004S|SET OF 3 SILVER FLYING DUCKS|1|2009-12-04 12:59:00|6.35|14527|United Kingdom
490353|18098C|PORCELAIN BUTTERFLY OIL BURNER|2|2009-12-04 16:37:00|2.95|15514|United Kingdom
491041|90013E|AMBER VINTAGE EARRINGS|1|2009-12-09 09:42:00|4.33|NULL|United Kingdom
490937|22086|PAPER CHAIN KIT 50'S CHRISTMAS|24|2009-12-08 13:58:00|2.95|13081|United Kingdom
490514|21784|SHOE SHINE BOX|1|2009-12-06 15:01:00|9.

In [55]:
from pymapd import connect
from time import time
import pandas as pd

In [47]:
con = connect(user="admin", password="HyperInteractive", host="localhost", dbname="omnisci")

In [38]:
start = time()
df = con.select_ipc_gpu("SELECT Invoice, SUM(Price) as price_sum FROM online_retail GROUP BY Invoice ORDER BY price_sum DESC LIMIT 10;") #<. un po' più veloce ma usa ipc
end = time()

print(end-start)

0.24244356155395508


In [52]:
start = time()
df = con.execute("SELECT Invoice, SUM(Price) as price_sum FROM online_retail GROUP BY Invoice ORDER BY price_sum DESC LIMIT 10;")
end = time()

print(end-start)

0.03366565704345703


In [56]:
pd.read_sql_query("SELECT Invoice, SUM(Price) as price_sum FROM online_retail GROUP BY Invoice ORDER BY price_sum DESC LIMIT 10;", con=con)

,Invoice,price_sum
0,C556445,38970.00
1,C537651,27082.66
2,537632,27082.66
3,C537630,27082.66
4,C537644,26949.58
5,512771,25111.09
6,C512770,25111.09
7,C520667,18910.69
8,C580605,17836.46
9,C540117,16888.02


In [57]:
%%bash
docker stop omnisci_test

omnisci_test
